# Document Classification with Naive Bayes

## Introduction

In this lesson, you'll investigate another implementation of the Bayesian framework in order to classify YouTube videos into the appropriate topic. The dataset you'll be investigating again comes from Kaggle. For further information, you can check out the original dataset here: https://www.kaggle.com/extralime/math-lectures .

## Objectives

You will be able to:  

- Implement document classification using Naive Bayes 
- Explain how to code a bag of words representation
- Explain why it is necessary to use Laplacian smoothing correction

## Bayes theorem for document classification

A common example of using Bayes' theorem to classify documents is a spam filtering algorithm. You'll be exploring this application in the upcoming lab. To do this, you examine the question "given this word (in the document) what is the probability that it is spam versus not spam?" For example, perhaps you get a lot of "special offer" spam. In that case, the words "special" and "offer" may increase the probability that a given message is spam.

Recall Bayes theorem:

 $$ \large  P(A|B) = \dfrac{P(B|A)P(A)}{P(B)}$$

Applied to a document, one common implementation of Bayes' theorem is to use a bag of words representation. A bag of words representation takes a text document and converts it into a word frequency representation. For example, in a bag of words representation, the message:

> "Thomas Bayes was born in the early 1700s, although his exact date of birth is unknown. As a Presbyterian in England, he took an unconventional approach to education for his day since Oxford and Cambridge were tied to the Church of England."

Would look like this:

In [1]:
doc = "Thomas Bayes was born in the early 1700s, although his exact date of birth is unknown. As a Presbyterian in England, he took an unconventional approach to education for his day since Oxford and Cambridge were tied to the Church of England."
bag = {}
for word in doc.split():
    # Get the previous entry, or 0 if not yet documented; add 1
    bag[word] = bag.get(word, 0) + 1 
bag

{'Thomas': 1,
 'Bayes': 1,
 'was': 1,
 'born': 1,
 'in': 2,
 'the': 2,
 'early': 1,
 '1700s,': 1,
 'although': 1,
 'his': 2,
 'exact': 1,
 'date': 1,
 'of': 2,
 'birth': 1,
 'is': 1,
 'unknown.': 1,
 'As': 1,
 'a': 1,
 'Presbyterian': 1,
 'England,': 1,
 'he': 1,
 'took': 1,
 'an': 1,
 'unconventional': 1,
 'approach': 1,
 'to': 2,
 'education': 1,
 'for': 1,
 'day': 1,
 'since': 1,
 'Oxford': 1,
 'and': 1,
 'Cambridge': 1,
 'were': 1,
 'tied': 1,
 'Church': 1,
 'England.': 1}

In [2]:
bag.get('England.',0)

1

In [7]:


bag.keys()
bag.values()

dict_values([1, 1, 1, 1, 2, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

Additional preprocessing techniques can also be applied to the document before applying a bag of words representation, many of which you'll explore later when further investigating Natural Language Processing (NLP) techniques. 


Once you've converted the document into a bag of words representation, you can then implement Bayes' theorem.
Returning to the case of 'Spam' and 'Not Spam', you would have:

 $$ P(\text{Spam | Word}) = \dfrac{P(\text{Word | Spam})P(\text{Spam})}{P(\text{Word})}$$  

Using the bag of words representation, you can then define $P(\text{Word | Spam})$ as

 $$P(\text{Word | Spam}) = \dfrac{\text{Word Frequency in Document}}{\text{Word Frequency Across All Spam Documents}}$$  

However, this formulation has a problem: what if you encounter a word in the test set that was not present in the training set? This new word would have a frequency of zero! This would commit two grave sins. First, there would be a division by zero error. Secondly, the numerator would also be zero; if you were to simply modify the denominator, having a term with zero probability would cause the probability for the entire document to also be zero when you subsequently multiplied the conditional probabilities in Multinomial Bayes. To effectively counteract these issues, Laplacian smoothing is often used giving:   

 $$P(\text{Word | Spam}) = \dfrac{\text{Word Frequency in Document} + 1}{\text{Word Frequency Across All Spam Documents + Number of Words in Corpus Vocabulary}}$$  

Now, to implement this in Python!

## Load the dataset

In [8]:
import pandas as pd
df = pd.read_csv('raw_text.csv')
df.head()

,text,label
0,The following content is\r\nprovided under a C...,Calculus
1,"In this sequence of segments,\r\nwe review som...",Probability
2,The following content is\r\nprovided under a C...,CS
3,The following\r\ncontent is provided under a C...,Algorithms
4,The following\r\ncontent is provided under a C...,Algorithms


In [9]:
print(df.iloc[1,0])

In this sequence of segments,
we review some mathematical background that will be
useful at various places in this course. Most of what is covered, with
the exception of the last segment, is material that you
may have seen before. But this could still be an
opportunity to refresh some of these concepts. I should say that
this is intended to be just a refresher. Our coverage is not going to
be complete in any sense. What we will talk about is
sets, various definitions related to sets, and some basic
properties, including De Morgan's laws. We will talk about what a
sequence is and what it means for a sequence to converge
to something. We will talk about
infinite series. And as an example, we will look
at the geometric series. Then we will talk about some
subtleties that arise when you have sums of terms that are
indexed with multiple indices. And finally, probably the most
sophisticated part, will be a discussion of countable versus
uncountable sets. Countable sets are like
the integers.

In [10]:
df['label'].value_counts()

Linear Algebra     152
Probability        124
CS                 104
Diff. Eq.           93
Algorithms          81
Statistics          79
Calculus            70
Data Structures     62
AI                  48
Math for Eng.       28
NLP                 19
Name: label, dtype: int64

## Simple two-class case

To simplify the problem, you can start by subsetting to two specific classes: 

In [11]:
df2 = df[df['label'].isin(['Algorithms', 'Statistics'])]
df2['label'].value_counts()

Algorithms    81
Statistics    79
Name: label, dtype: int64

In [13]:
df2.head()

,text,label
3,The following\r\ncontent is provided under a C...,Algorithms
4,The following\r\ncontent is provided under a C...,Algorithms
6,The following content is\r\nprovided under a C...,Algorithms
16,The following content is\r\nprovided under a C...,Algorithms
18,The following content is\r\nprovided under a C...,Algorithms


In [14]:
df.shape,df2.shape

((860, 2), (160, 2))

In [15]:
df['label'].isin(['Algorithms', 'Statistics'])

0      False
1      False
2      False
3       True
4       True
       ...  
855    False
856    False
857    False
858    False
859    False
Name: label, Length: 860, dtype: bool

In [16]:
df2['label'].value_counts(normalize=True)


Algorithms    0.50625
Statistics    0.49375
Name: label, dtype: float64

In [18]:
p_classes = dict(df2['label'].value_counts(normalize=True))
p_classes

{'Algorithms': 0.50625, 'Statistics': 0.49375}

In [19]:
df2.iloc[0]

text     The following\r\ncontent is provided under a C...
label                                           Algorithms
Name: 3, dtype: object

## Train-test split

In [20]:
from sklearn.model_selection import train_test_split
X = df2['text']
y = df2['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=17)
train_df = pd.concat([X_train, y_train], axis=1) 
test_df = pd.concat([X_test, y_test], axis=1)

In [21]:
train_df.head()

,text,label
196,"In our previous lesson, we saw what binary\r\n...",Algorithms
729,The following content is\r\nprovided under a C...,Statistics
261,"OK, here we go with, quiz\r\nreview for the th...",Algorithms
185,A lot of what we do with Laplace\r\ntransforms...,Statistics
294,Let us now discuss an\r\ninteresting fact abou...,Statistics


In [22]:
test_df.head()

,text,label
685,OPERATOR: -- The following content is\r\nprovi...,Algorithms
375,[SOUND] Stanford University. &gt;&gt; We'll ge...,Statistics
338,"Okay, this is linear\r\nalgebra, lecture four....",Statistics
80,The following content is\r\nprovided under a C...,Algorithms
828,The following content is\r\nprovided under a C...,Statistics


## Create the word frequency dictionary for each class

In [23]:
# Will be a nested dictionary of class_i : {word1:freq, word2:freq..., wordn:freq},.... class_m : {}
class_word_freq = {} 
classes = train_df['label'].unique()
for class_ in classes:
    temp_df = train_df[train_df.label == class_]
    bag = {}
    for row in temp_df.index:
        doc = temp_df['text'][row]
        for word in doc.split():
            bag[word] = bag.get(word, 0) + 1
    class_word_freq[class_] = bag

In [35]:
print(temp_df.index)
temp_df['text'][729].split()


Int64Index([729, 185, 294, 400, 677, 365, 599, 538, 481, 547, 822, 537, 281,
            654, 533, 221, 277, 201, 690, 616, 136,  73, 309, 495, 239, 323,
            476, 298, 432, 142,  58, 115, 839, 780, 465, 456,  77, 112, 364,
            440, 561, 838, 750, 460, 549, 437, 273,  46, 841, 431, 286, 234,
            808,  23, 316, 198, 129, 291, 778],
           dtype='int64')


['The',
 'following',
 'content',
 'is',
 'provided',
 'under',
 'a',
 'Creative',
 'Commons',
 'license.',
 'Your',
 'support',
 'will',
 'help',
 'MIT',
 'OpenCourseWare',
 'continue',
 'to',
 'offer',
 'high',
 'quality',
 'educational',
 'resources',
 'for',
 'free.',
 'To',
 'make',
 'a',
 'donation',
 'or',
 'view',
 'additional',
 'materials',
 'from',
 'hundreds',
 'of',
 'MIT',
 'courses,',
 'visit',
 'MIT',
 'OpenCourseWare',
 'at',
 'ocw.mit.edu.',
 'PROFESSOR:',
 'OK.',
 'So',
 "let's",
 'get',
 'started.',
 'We',
 'want',
 'to',
 'first',
 'review',
 "Wald's",
 'equality',
 'a',
 'little',
 'bit.',
 "Wald's",
 'equality',
 'is',
 'very',
 'tricky',
 'thing.',
 'If',
 'you',
 'think',
 'you',
 'understand',
 'it,',
 'you',
 'will',
 'go',
 'along.',
 'And',
 'at',
 'some',
 'point,',
 'you',
 'will',
 'be',
 'using',
 'it',
 'and',
 'you',
 'will',
 'say,',
 'I',
 "don't",
 'understand',
 'what',
 'this',
 'says.',
 'And',
 'that',
 'will',
 'happen',
 'for',
 'a',
 'long',

In [24]:
print(train_df['label'].unique())

['Algorithms' 'Statistics']


In [27]:
train_df [train_df.label=='Algorithms']

,text,label
196,"In our previous lesson, we saw what binary\r\n...",Algorithms
261,"OK, here we go with, quiz\r\nreview for the th...",Algorithms
434,We have already seen\r\nan example in which we...,Algorithms
131,"So, we're gonna start on our very\r\nlast topi...",Algorithms
647,A lot of what you'll learn in\r\ndifferential ...,Algorithms
...,...,...
355,The following content is\r\nprovided under a C...,Algorithms
167,[MUSIC] Stanford University. &gt;&gt; Alright!...,Algorithms
821,The following content is\r\nprovided under a C...,Algorithms
716,Let us now study a very\r\nimportant counting ...,Algorithms


In [40]:
class_word_freq.keys()
class_word_freq['Algorithms'].keys()
class_word_freq['Algorithms'].values()


dict_values([309, 609, 64, 3, 4289, 32, 1839, 71, 75, 9, 38, 686, 4298, 5509, 11, 2120, 2018, 10307, 34, 34, 678, 1337, 2315, 43, 844, 68, 2262, 3, 8286, 6460, 5846, 63, 94, 17404, 12, 8973, 7, 5719, 16, 83, 2, 1, 602, 3137, 137, 19, 15, 110, 6, 1294, 40, 25, 1381, 12, 72, 1169, 100, 1300, 707, 27, 70, 4, 771, 1503, 61, 7, 1814, 20, 1317, 12, 2, 3, 583, 7279, 1289, 404, 25, 396, 1318, 149, 15, 9, 543, 325, 4, 1800, 103, 40, 49, 860, 301, 3, 87, 92, 1689, 177, 47, 445, 94, 506, 183, 2, 412, 11, 4571, 14, 9, 631, 387, 203, 292, 697, 86, 7, 15, 2, 17, 297, 63, 877, 7, 762, 6, 917, 177, 9, 448, 37, 6, 34, 583, 278, 3, 1772, 3178, 12, 243, 27, 42, 161, 1708, 305, 5, 98, 1124, 17, 1531, 116, 9, 493, 1127, 29, 38, 74, 23, 246, 108, 9, 15, 7, 564, 250, 279, 5, 93, 1, 117, 33, 53, 243, 75, 13, 877, 20, 1479, 53, 17, 6, 209, 686, 86, 3, 44, 21, 139, 25, 851, 234, 698, 227, 1, 447, 76, 1, 10, 699, 8, 1, 11, 22, 3, 14, 406, 1, 174, 10, 2, 519, 9, 391, 11, 51, 1, 497, 575, 4, 267, 7, 71, 246, 3, 18

## Count the total corpus words

In [9]:
vocabulary = set()
for text in train_df['text']:
    for word in text.split():
        vocabulary.add(word)
V = len(vocabulary)
V

23977

## Create a bag of words function

In [10]:
def bag_it(doc):
    bag = {}
    for word in doc.split():
        bag[word] = bag.get(word, 0) + 1
    return bag

## Implement Naive Bayes

In [11]:
import numpy as np
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = p_classes[class_]
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p *= (num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

In [12]:
classify_doc(train_df.iloc[0]['text'], class_word_freq, p_classes, V, return_posteriors=True)

[0.0, 0.0]


'Algorithms'

## Avoid underflow

As you can see from the output above, repeatedly multiplying small probabilities can lead to underflow; rounding to zero due to numerical approximation limitations. As such, a common alternative is to add the logarithms of the probabilities as opposed to multiplying the raw probabilities themselves. If this is alien to you, it might be worth reviewing some algebra rules of exponents and logarithms briefly:  

$ e^x \cdot e^y = e^{x+y}$  
$ log_{e}(e)=1 $  
$ e^{log(x)} = x$  

With that, here's an updated version of the function using log probabilities to avoid underflow:

In [13]:
def classify_doc(doc, class_word_freq, p_classes, V, return_posteriors=False):
    bag = bag_it(doc)
    classes = []
    posteriors = []
    for class_ in class_word_freq.keys():
        p = np.log(p_classes[class_])
        for word in bag.keys():
            num = bag[word]+1
            denom = class_word_freq[class_].get(word, 0) + V
            p += np.log(num/denom)
        classes.append(class_)
        posteriors.append(p)
    if return_posteriors:
        print(posteriors)
    return classes[np.argmax(posteriors)]

In [14]:
classify_doc(train_df.iloc[0]['text'], class_word_freq, p_classes, V, return_posteriors=True)

[-5578.267536771343, -5577.213285866603]


'Statistics'

In [15]:
classify_doc(train_df.iloc[10]['text'], class_word_freq, p_classes, V, return_posteriors=True)

[-2572.1544445158343, -2571.311308656896]


'Statistics'

In [16]:
classify_doc(train_df.iloc[12]['text'], class_word_freq, p_classes, V, return_posteriors=True)

[-4602.602622507951, -4601.755644621728]


'Statistics'

In [17]:
y_hat_train = X_train.map(lambda x: classify_doc(x, class_word_freq, p_classes, V))
residuals = y_train == y_hat_train
residuals.value_counts(normalize=True)

False    0.508333
True     0.491667
dtype: float64

As you can see, this algorithm leaves a lot to be desired. A measly 49% accuracy is nothing to write home about. (In fact, it's slightly worse than random guessing!) In practice, substantial additional preprocessing including removing stop words and using stemming or lemmatisation would be required. Even then, Naive Bayes might still not be the optimal algorithm. Nonetheless, it is a worthwhile exercise and a comprehendible algorithm. 

## Summary

In this lesson, you got to see another application of Bayes' theorem as a means to do some rough documentation classification.